In [1]:
import sys
sys.path.append("/Volumes/pdoubraw-1/repos/ABLTools/python/")
sys.path.append("C:/Users/krydalch/Documents/Repos/ABLtools_python/")
import numpy as np
import matplotlib.pyplot as plt
import stochasticTurbulenceTools as stt
import glob, os
import pandas as pd
import functions as f

sys.path.append("C:/Users/krydalch/Documents/Repos/")
from datatools import wfip2
from datatools.remote_sensing import scintec_profiler


plt.rc("font",size=14)

# Input Data sets: (TurbSim.bts, UTS_input_file, True Sonic Data)

In [ ]:
kir_ts = stt.stochasticTurbulence("TurbSim")
#kir_ts.readBTS("/Users/pdoubraw/Downloads/")
kir_ts.readBTS("C:/Users/krydalch/Desktop/TurbSim/MyOutputFiles/LLJ/1440_1450_rand1_USR2pts/")

In [ ]:
one_USR = stt.stochasticTurbulence("TurbSim")
#kir_ts.readBTS("/Users/pdoubraw/Downloads/")
one_USR.readBTS("C:/Users/krydalch/Desktop/TurbSim/MyOutputFiles/LLJ/1440_1450_rand1_USR1pts/")

In [2]:
all_USR = stt.stochasticTurbulence("TurbSim")
#kir_ts.readBTS("/Users/pdoubraw/Downloads/")
all_USR.readBTS("C:/Users/krydalch/Desktop/TurbSim/MyOutputFiles/LLJ/1440_1450_rand1_USRall/")

Opening file C:/Users/krydalch/Desktop/TurbSim/MyOutputFiles/LLJ/1440_1450_rand1_USRall/TurbSim.bts...


error: unpack requires a buffer of 16 bytes

In [ ]:
df_sonic = pd.read_csv('UTS_1440_1450.txt', sep='\t', header=None)

In [ ]:
path_to_csvs   = "C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\"
start_time = '14:40:00'
end_time = '14:50:00'
master_df = f.get_sonic_met_dat(path_to_csvs, start_time ,end_time)

In [ ]:
heights_in_USR = [116.4, 158.2]
u ={}
v = {}
w = {}

ti =[]

#kir_ts
us = range(1,5,3)
heights = [116.4, 158.2]
time = df_sonic.loc[:,0]
i = 0
###########3 Read in Text File #################3
for u_h in us:
    u[heights[i]] = df_sonic[u_h]
    v[heights[i]] = df_sonic[u_h+1]
    w[heights[i]] = df_sonic[u_h+2]
    
    uu = np.sqrt((df_sonic[u_h] - df_sonic[u_h].mean())**2)
    
    ti_s = (uu/ df_sonic[u_h].mean()).mean()

    ti = np.append(ti,ti_s)
    i = i +1

########### TurbSim 1 ################
TI_turb = kir_ts.TI()
Ti_hub = TI_turb[kir_ts.jHub, :]
########### TurbSim 1 ################
TI_turb_2 = one_USR.TI()
Ti_hub_2 = TI_turb_2[one_USR.jHub, :]
# ########### TurbSim 1 ################
# TI_turb_3 = all_USR .TI()
# Ti_hub_3 = TI_turb_3[all_USR.jHub, :]

###########3 True Sonic Data ###########3
temp_df = master_df.copy()
TI_master= []
heights_mstr = [0.9, 2.4, 4.0, 10.1, 16.8, 74.7, 116.4, 158.2, 200.0]
for h in heights_mstr:
    u_m = temp_df[h]['U']
    uu_m = np.sqrt((u_m - u_m.mean())**2)
    ti_m = (uu_m / u_m.mean()).mean()
    
    TI_master = np.append(TI_master,ti_m)

########################################3



fig = plt.figure(figsize=(12,9))
plt.plot(Ti_hub_2,one_USR.z,label = 'TurbSim, 1 points[115.0]')
#plt.plot(Ti_hub_3,all_USR.z,label = 'TurbSim, 4 points[16.8, 47.2, 115.0, 158.2, 200]')
plt.plot(Ti_hub,kir_ts.z,label = 'TurbSim, 2 points[115.0, 158.2]')
#plt.plot(ti,heights,label= 'UTS')
plt.plot(TI_master,heights_mstr,'p--k', label ='Sonic Raw')

plt.legend()
plt.title("TI average throughout simulation")
plt.xlabel("TI")
plt.ylabel("Z [m]")
fig.savefig("../TurbSim_UTSinputs_one.png",bbox_inches="tight",dpi=500)